In [32]:
# Primero ejecutar el archivo 05_limpiar_datos.pandas.ipynb antes que este. Es importante!!!

import pandas as pd
import os
import re


In [33]:
archivo_1 = 'excels/jugadores_nba.xlsx'
archivo_2 = 'excels/actualizados/estadistica_jugadores_nba_actualizado.xlsx'

df1 = pd.read_excel(archivo_1)
df2 = pd.read_excel(archivo_2)

In [34]:
print(df1.columns)
print(df2.columns)

Index(['id_jugador', 'Nombre', 'Apellido', 'Equipo', 'Dorsal', 'Posición',
       'PPG', 'RPG', 'APG', 'PIE', 'Altura', 'Peso', 'País', 'Último Equipo',
       'Edad', 'Cumpleaños', 'Draft', 'Experiencia'],
      dtype='object')
Index(['id_jugador', 'Player', 'ID Equipo', 'GP', 'MIN', 'PTS', 'FGM', 'FGA',
       'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'EFF', 'AST/TOV', 'STL/TOV'],
      dtype='object')


In [35]:
df1.drop('Equipo', axis=1, inplace=True)
df2.drop(['Player'], axis=1, inplace=True)

In [36]:
# Vamos a limpiar la columna dorsal. Vamos a quitar las almohadillas y si hay algún valor que no sea numérico o está en blanco ponerle none

df1['Dorsal'] = df1['Dorsal'].str.replace('#', '', regex=True)
df1['Dorsal'] = df1['Dorsal'].apply(lambda x: None if not str(x).isdigit() else x)
print(df1['Dorsal'])

0       5
1       4
2      13
3      30
4       7
       ..
538    21
539    11
540    77
541    40
542    40
Name: Dorsal, Length: 543, dtype: object


In [37]:
conteo_posiciones = df1['Posición'].value_counts()
valores_blancos = df1['Posición'].isna().sum()
print("Conteo de Posiciones:\n", conteo_posiciones)
print("\nCantidad de valores en blanco:", valores_blancos)

Conteo de Posiciones:
 Posición
Guard             207
Forward           160
Center             44
Guard-Forward      39
Forward-Center     32
Center-Forward     21
Forward-Guard      16
none                3
Name: count, dtype: int64

Cantidad de valores en blanco: 21


In [38]:
# Voy a cambiar el nombre de las posiciones a español
# Primero creamos un diccionario
posiciones_español = {
    'Guard': 'Base',
    'Forward': 'Alero',
    'Center': 'Pivot',
    'Forward-Center': 'Ala-Pivot',
    'Guard-Forward': 'Base - Alero',
    'Forward-Guard': 'Alero-Escolta',
    'Center-Forward': 'Ala-Pivot'
}

df1['Posición'] = df1['Posición'].replace(posiciones_español)

# Y ahora en los sitios que esté blanco o no haya datos se le pone none en texto
df1['Posición'] = df1['Posición'].apply(lambda x: 'none' if x not in posiciones_español.values() else x)
print(df1['Posición'])

0          Alero
1          Pivot
2      Ala-Pivot
3           Base
4      Ala-Pivot
         ...    
538        Alero
539         Base
540        Pivot
541    Ala-Pivot
542        Pivot
Name: Posición, Length: 543, dtype: object


In [39]:
# Ahora vamos a trabajar a la vez con las cuatro columnas numéricas de datos del dataframe

columnas = ['PPG', 'RPG', 'APG', 'PIE']

# La idea es que si no tiene número que me ponga none en texto
def valores_numero(valor):
    try:
        return pd.to_numeric(valor)
    except ValueError:
        return 'none'

for columna in columnas:
    df1[columna] = df1[columna].apply(valores_numero)

print(df1['PPG'])
print(df1['RPG'])
print(df1['APG'])
print(df1['PIE'])

0       6.9
1       8.6
2      22.0
3       6.2
4      10.4
       ... 
538     2.9
539    27.8
540     3.8
541     2.0
542    12.6
Name: PPG, Length: 543, dtype: object
0       5.1
1      11.5
2      10.3
3       2.5
4       5.5
       ... 
538     1.3
539     3.1
540     4.5
541     2.8
542     9.7
Name: RPG, Length: 543, dtype: object
0       1.5
1       2.3
2       3.9
3       1.0
4       1.7
       ... 
538     1.2
539    11.0
540     0.6
541     1.1
542     1.3
Name: APG, Length: 543, dtype: object
0       9.6
1      11.2
2      15.8
3       5.5
4      10.7
       ... 
538    10.2
539    14.2
540     8.2
541     6.1
542    12.4
Name: PIE, Length: 543, dtype: object


In [40]:
# Vamos ahora a trabjar con la columna altura. Lo primero es cambiarle el nombre para que ponga Altura (metros) para saber ese dato
# Y luego la idea es dejar solo el valor en metros del dato. Utilizando expresiones regulares

df1.rename(columns={'Altura': 'Altura (metros)'}, inplace=True)

def extraer_altura_metros(valor):
    if pd.isna(valor):
        return None
    try:
        # Utilizamos una expresión regular
        altura_metros = pd.Series(valor).str.extract(r'\((\d+\.\d+)m\)')[0]
        return altura_metros
    except Exception:
        return 'none'


df1['Altura (metros)'] = df1['Altura (metros)'].apply(extraer_altura_metros)
print(df1['Altura (metros)'])


0      2.03
1      2.11
2      2.06
3      1.96
4      2.13
       ... 
538    2.03
539    1.85
540    2.11
541    2.11
542    2.13
Name: Altura (metros), Length: 543, dtype: object


In [41]:
import pandas as pd

# Cambiar el nombre de la columna 'Peso' a 'Peso (kg)'
df1.rename(columns={'Peso': 'Peso (kg)'}, inplace=True)

# Función para extraer el valor en kilogramos
def extraer_peso_kg(valor):
    if pd.isna(valor):
        return 'none'
    try:
        # Extraer la parte en kilogramos utilizando una expresión regular
        peso_kg = pd.Series(valor).str.extract(r'\((\d+)kg\)')[0]
        return peso_kg
    except Exception:
        return 'none'

# Aplicar la función a la columna 'Peso (kg)'
df1['Peso (kg)'] = df1['Peso (kg)'].apply(extraer_peso_kg)

print(df1['Peso (kg)'])

0      110
1      120
2      116
3       98
4       98
      ... 
538    102
539     74
540    125
541    109
542    109
Name: Peso (kg), Length: 543, dtype: object


In [42]:
# Función para limpiar la columna Edad
def limpiar_edad(valor):
    if pd.isna(valor) or not str(valor).endswith("years"):
        return None
    return str(valor).replace("years", "").strip()

# Aplicar la función a la columna Edad
df1['Edad'] = df1['Edad'].apply(limpiar_edad)

# Función para limpiar la columna Experiencia
def limpiar_experiencia(valor):
    if pd.isna(valor):
        return None
    if valor == "Rookie":
        return valor
    if str(valor).lower().endswith("years") or str(valor).lower().endswith("year"):
        return str(valor).replace("Years", "").replace("years", "").replace("Year", "").replace("year", "").strip()
    return None

# Aplicar la función a la columna Experiencia
df1['Experiencia'] = df1['Experiencia'].apply(limpiar_experiencia)

print(df1['Edad'])
print(df1['Experiencia'])

0      24
1      30
2      26
3      23
4      23
       ..
538    35
539    25
540    25
541    31
542    26
Name: Edad, Length: 543, dtype: object
0       3
1      10
2       6
3       1
4       2
       ..
538    16
539     5
540     2
541    10
542     7
Name: Experiencia, Length: 543, dtype: object


In [43]:
# Ahora vamos con la columna cumpleaños, para ponerla en formato fecha más reconocible en España

df1['Cumpleaños'] = pd.to_datetime(df1['Cumpleaños'], errors='coerce').dt.strftime('%d-%m-%Y')

df1['Cumpleaños'] = df1['Cumpleaños'].where(df1['Cumpleaños'].notna(), None)


In [44]:
print(df1['Cumpleaños'])

0      19-09-1999
1      20-07-1993
2      18-07-1997
3      20-04-2000
4      10-01-2001
          ...    
538    21-06-1988
539    19-09-1998
540    19-06-1998
541    05-10-1992
542    18-03-1997
Name: Cumpleaños, Length: 543, dtype: object


In [48]:

# Fusionar df2 con df1 usando 'id_jugador' como clave, preservando todos los registros de df2
df_jugadores = pd.merge(df2, df1, on='id_jugador', how='left', suffixes=('', '_df1'))

# Lista del orden deseado de columnas
orden_columnas = ['id_jugador', 'Nombre', 'Apellido', 'ID Equipo'] + \
                 [col for col in df1.columns if col not in ['id_jugador', 'Nombre', 'Apellido']] + \
                 [col for col in df2.columns if col not in ['id_jugador', 'ID Equipo']]

# Reordenar las columnas en df_jugadores
df_jugadores = df_jugadores[orden_columnas]

# Rellenar con None los valores NaN resultantes de la fusión
df_jugadores.fillna('None', inplace=True)



In [50]:
ruta_archivo = 'excels/actualizados/jugadores_completos.xlsx'

df_jugadores.to_excel(ruta_archivo, index=False)
